In [31]:
import os, json, asyncio
from pathlib import Path
from playwright.async_api import async_playwright
import pandas as pd
from IPython.display import clear_output

In [8]:
df = pd.read_parquet("datos/2025.parquet")

In [32]:
nro_documentos = df.nro_documento.tolist()

In [37]:
CHECKPOINT = Path("sigep_checkpoint.json")

In [40]:
async def run(nro_documentos):
    async with async_playwright() as p:
        if CHECKPOINT.exists():
            with open(CHECKPOINT, "r") as f:
                done, data = json.load(f)
        else:
            done, data = [], []

        remaining = [c for c in nro_documentos if c not in done]
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(
            "https://sigep.gob.bo/rsportaltransparencia/#/portalHome/personal"
        )
        await page.wait_for_load_state("networkidle")
        await page.click("a.pull-right[type=button]")
        for nro_documento in remaining:
            clear_output(wait=True)
            print(f"{len(done)} / {len(nro_documentos)} documentos ({len(data)} registros): {nro_documento}")
            try:
                await page.fill("input[name=ci]", str(nro_documento))
                async with page.expect_response(
                    "**/persona/lista?tipoBusqueda=AP&**", timeout=30000
                ) as response_info:
                    await page.click("input[value=Buscar]")
                response = await response_info.value
                if response:
                    response_data = await response.json()
                if response_data["data"]["lista"]:
                    data.append(
                        {
                            **{"nro_documento": nro_documento},
                            **response_data["data"]["lista"][0],
                        }
                    )
                done = done + [nro_documento]
                await asyncio.sleep(2)
            except Exception as e:
                print(e)
                break
            with open(CHECKPOINT, "w") as f:
                json.dump([done, data], f, ensure_ascii=False, indent=2)

        await browser.close()

In [41]:
await run(nro_documentos)

1863 / 1864 documentos (191 registros): 3836306


In [44]:
with open(CHECKPOINT, 'r') as f:
    sigep = pd.DataFrame(json.load(f)[1])

In [45]:
sigep

,nro_documento,id,memorandum,entidad,idEntidad,siglaEntidad,descEntidad,descUrh,descUnidad,nombres,primerApellido,segundoApellido,razonSocial,descCargo,descPuesto,descModalidad,monto,fechaIni,fechaFin,apiEstado
0,7971986,390688,390,650,575,ALP,Asamblea Legislativa Plurinacional,CAMARA DE SENADORES,SENADORES TITULARES BRIGADA COCHABAMBA,ANDRONICO,RODRIGUEZ,LEDEZMA,RODRIGUEZ LEDEZMA ANDRONICO,SENADORES TITULARES,SENADOR TITULAR,PERSONAL DE PLANTA,23311.00,01/05/2024,None,Vigente
1,10667840,391054,334,169,403,AIT,Autoridad de Impugnación Tributaria,RRHH AIT,SECRETARIA DE CAMARA,CARLA FAVIOLA,FLORES,AGUIRRE,FLORES AGUIRRE CARLA FAVIOLA,ANALISTA I - SECRETARIO CÁMARA II,ANALISTA DE INFORMACION LEGAL Y DILIGENCIAS,PERSONAL DE PLANTA,13049.00,01/05/2024,None,Vigente
2,6164154,365509,286,680,579,CGE,Contraloria General del Estado,GNRH CONTRALORÍA GENERAL DEL ESTADO,SUBCONTRALORÍA DE SERVICIOS LEGALES,SANDRA,HINOJOSA,NUÑEZ,HINOJOSA NUÑEZ SANDRA,AUXILIAR I,AUXILIAR,PERSONAL DE PLANTA,5263.00,01/05/2024,None,Vigente
3,6180723,428952,4069,292,487,VOBO,Vías Bolivia,URH VIAS BOLIVIA OFICINA NACIONAL,OFICINA REGIONAL LA PAZ,HERLAN PAOOLO,None,PACHECO,PACHECO HERLAN PAOOLO,TÉCNICO IV,CHOFER LA PAZ,PERSONAL DE PLANTA,4469.00,01/05/2024,None,Vigente
4,7061347,576244,3139,340,358,SEGIP,Servicio General de Identificación Personal,URH - ENTIDAD 340 SEGIP,DIRECCION NACIONAL DE COMUNICACION,PAOLA,HUARACHI,MAMANI,HUARACHI MAMANI PAOLA,TECNICO I,TECNICO COMUNICACION,PERSONAL DE PLANTA,5729.00,01/05/2024,None,Vigente
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,4082480,362882,2296,650,575,ALP,Asamblea Legislativa Plurinacional,CAMARA DE DIPUTADOS,BRIGADAS,FERNANDO CARLOS,GUTIERREZ,ESPINOZA,GUTIERREZ ESPINOZA FERNANDO CARLOS,JEFE DE DIVISION,SECRETARIO TECNICO,PERSONAL DE PLANTA,13134.00,01/05/2024,None,Vigente
187,2406103,527401,3132,1205,50,ELLA,Gobierno Autónomo Municipal de El Alto de La Paz,ADMINISTRACION CENTRAL - EJECUTIVO,UNIDAD DE PROGRAMAS DE SEGURIDAD CIUDADANA Y S...,VICTOR,HUARANCA,MAMANI,HUARANCA MAMANI VICTOR,PROFESIONAL F,ASISTENTE ADMINISTRATIVO,PERSONAL DE PLANTA,4521.00,01/01/2024,None,Vigente
188,4262823,527516,66,1224,69,AHI,Gobierno Autónomo Municipal de Achacachi,URH GOBIERNO AUTÓNOMO MUNICIPAL DE ACHACACHI,DESPACHO ALCALDE,CONSTANCIO,GUTIERREZ,CATACORA,GUTIERREZ CATACORA CONSTANCIO,ALCALDE MUNICIPAL,ALCALDE,PERSONAL DE PLANTA,13638.23,01/01/2024,None,Vigente
189,6973816,472464,404,902,453,GAD-LPZ,Gobierno Autónomo Departamental de La Paz,ASAMBLEA LEGISLATIVA DEPARTAMENTAL,PLENO DE LA ASAMBLEA DEPARTAMENTAL DE LA PAZ,WALTER,PINTO,MOLLINEDO,PINTO MOLLINEDO WALTER,ASAMBLEISTA DEPARTAMENTAL,ASAMBLEISTA,PERSONAL DE PLANTA,14869.00,01/01/2023,None,Vigente


In [46]:
sigep.to_parquet("datos/sigep.parquet")